In [2]:
import os, re, magic, shutil
from glob import glob
import time, datetime
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import joblib
import datetime as dt

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import torch, gc
from torch import nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import torchvision
import torchvision.models as models

#from skimage import io
import sklearn
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss, f1_score, confusion_matrix, classification_report
from sklearn import metrics, preprocessing
from scipy.ndimage import zoom
import timm
import albumentations as A
import albumentations.pytorch
import wandb

In [3]:
CFG = {
    'seed': 42,
    'model': 'inception_resnet_v2',
    'bs':128,
    'num_workers': 10,
    'device': 'cuda:0',
}

##### Pandas dataframe - disaster, non_disaster 폴더에서 이미지 경로를 전부 가져와 frame에 저장
---
##### frame엔 image_id: 이미지 이름 // dir: 이미지 경로 // label 을 포함

In [4]:
infer_path = "../Data/disaster/test/" ## 데이터 루트 경로
label_list = ["disaster","non_disaster"] ## 폴더 (레이블) 이름

infer_img_paths = []
infer_img_labels = []

for label in label_list: ## 각 폴더 (레이블) 돌기
    print(f'label: {label}',end=' ')
    img_paths = [] 
    img_labels = []
    dir_path = infer_path + label ## 레이블 폴더 경로

    for folder, subfolders, filenames in os.walk(dir_path): ## 폴더 내 모든 파일 탐색
        for img in filenames: ## 각 파일 경로, 레이블 저장
            img_paths.append(folder+'/'+img)
            img_labels.append(label)
    
    print(len(img_paths))

    infer_img_paths.extend(img_paths)
    infer_img_labels.extend(img_labels)

print('infer_Images: ',len(infer_img_paths))
print("infer_Images_labels:", len(infer_img_labels))

label: disaster 400
label: non_disaster 20000
infer_Images:  20400
infer_Images_labels: 20400


In [5]:
## Pandas Dataframe
infer_df = pd.DataFrame(infer_img_paths, columns=['image_id'])
infer_df['dir'] = infer_df['image_id'].apply(lambda x: os.path.dirname(x))
infer_df['image_id'] = infer_df['image_id'].apply(lambda x: os.path.basename(x))
infer_df['label'] = infer_img_labels
## Encoding test data labels
infer = infer_df
infer

,image_id,dir,label
0,a8271e8c2fa02700.jpg,../Data/disaster/test/disaster/origin_images/o...,disaster
1,bb92b3c52eb018c8.jpg,../Data/disaster/test/disaster/origin_images/o...,disaster
2,cbf5687e145dc19f.jpg,../Data/disaster/test/disaster/origin_images/o...,disaster
3,54703d04c4271a5a.jpg,../Data/disaster/test/disaster/origin_images/o...,disaster
4,a07b85e2b03909c1.jpg,../Data/disaster/test/disaster/origin_images/o...,disaster
...,...,...,...
20395,0b61646f2795ca9b.jpg,../Data/disaster/test/non_disaster/open_images,non_disaster
20396,0b16c09057d87676.jpg,../Data/disaster/test/non_disaster/open_images,non_disaster
20397,07ddddad35140b20.jpg,../Data/disaster/test/non_disaster/open_images,non_disaster
20398,0143bd9e146f6ded.jpg,../Data/disaster/test/non_disaster/open_images,non_disaster


In [6]:
## Label encoding (0: non-disaster ~ 1: disaster)
le = preprocessing.LabelEncoder()
infer['label'] = le.fit_transform(infer['label'].values)
infer

,image_id,dir,label
0,a8271e8c2fa02700.jpg,../Data/disaster/test/disaster/origin_images/o...,0
1,bb92b3c52eb018c8.jpg,../Data/disaster/test/disaster/origin_images/o...,0
2,cbf5687e145dc19f.jpg,../Data/disaster/test/disaster/origin_images/o...,0
3,54703d04c4271a5a.jpg,../Data/disaster/test/disaster/origin_images/o...,0
4,a07b85e2b03909c1.jpg,../Data/disaster/test/disaster/origin_images/o...,0
...,...,...,...
20395,0b61646f2795ca9b.jpg,../Data/disaster/test/non_disaster/open_images,1
20396,0b16c09057d87676.jpg,../Data/disaster/test/non_disaster/open_images,1
20397,07ddddad35140b20.jpg,../Data/disaster/test/non_disaster/open_images,1
20398,0143bd9e146f6ded.jpg,../Data/disaster/test/non_disaster/open_images,1


In [7]:
"""랜덤 수의 Seed 고정, 재현성 보장"""
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [8]:
def get_img(path, sub_path=None):
    try:
        im_bgr = cv2.imread(path)
        im_rgb = im_bgr[:, :, ::-1]
        past_path = path
    except: ## 이미지 에러 발생 시 백지로 대체
        #im_bgr = cv2.imread('../Data/carbon_reduction/temp_img.jpg')
        im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

In [9]:
transform = A.Compose(
    [
        A.Resize(height = 256, width = 256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.pytorch.transforms.ToTensorV2()

        ])

In [10]:
class CustomDataset(Dataset):
    def __init__(self, df, data_root, transform=None, output_label=True):
        super(CustomDataset,self).__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transform = transform
        self.data_root = data_root
        self.output_label = output_label
         
        if output_label == True:
            self.labels = self.df['label'].values
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # GET IMAGES
        path = "{}/{}".format(self.data_root[index], self.df.iloc[index]['image_id'])
        img  = get_img(path)
        
        # GET LABELS
        if self.output_label:
            target = self.labels[index]
            transformed =self.transform(image=img)
            img = transformed['image']
            return img, target
        else:
            transformed =self.transform(image=img)
            img = transformed['image']
            return img

In [11]:
class baseModel(nn.Module):
    def __init__(self, model_arch, n_class=2, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained, num_classes=n_class)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [12]:
########################## inference #############################
def inference(model, data_loader, device):
    model.eval()
    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()

        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [15]:
""" 이미지 분석을 위한 기본 전처리 """
transform_train = A.Compose(
    [
        A.Resize(height = 256, width = 256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.pytorch.transforms.ToTensorV2()
        ])

In [16]:
def load_img(image_path):
    input_image = Image.open(image_path).convert('RGB')
    input_tensor = transform(input_image)
    return input_tensor.unsqueeze(0)

In [21]:
# RUN INFERENCE
model = baseModel(CFG['model'], infer.label.nunique(), pretrained=True)
load_model = './models/inception_resnet_v2.pth'
infer_dir = infer.dir.values

infer_ds = CustomDataset(infer, infer_dir, transform=transform, output_label=False)
infer_loader = torch.utils.data.DataLoader(
    infer_ds, 
    batch_size=CFG['bs'],
    num_workers=CFG['num_workers'],
    shuffle=False,
    pin_memory=True
)
device = torch.device(CFG['device'])

#INFERENCE VIA MULTI-GPU
# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model)
model.to(device)

# RUN INFERENCE
predictions = []
model.load_state_dict(torch.load(load_model))
with torch.no_grad():
    predictions += [inference(model, infer_loader, device)]


predictions = np.mean(predictions, axis=0) 
infer['pred'] = np.argmax(predictions, axis=1)
infer

100%|██████████| 160/160 [00:57<00:00,  2.77it/s]


,image_id,dir,label,pred
0,a8271e8c2fa02700.jpg,../Data/disaster/test/disaster/origin_images/o...,0,1
1,bb92b3c52eb018c8.jpg,../Data/disaster/test/disaster/origin_images/o...,0,0
2,cbf5687e145dc19f.jpg,../Data/disaster/test/disaster/origin_images/o...,0,0
3,54703d04c4271a5a.jpg,../Data/disaster/test/disaster/origin_images/o...,0,1
4,a07b85e2b03909c1.jpg,../Data/disaster/test/disaster/origin_images/o...,0,0
...,...,...,...,...
20395,0b61646f2795ca9b.jpg,../Data/disaster/test/non_disaster/open_images,1,1
20396,0b16c09057d87676.jpg,../Data/disaster/test/non_disaster/open_images,1,1
20397,07ddddad35140b20.jpg,../Data/disaster/test/non_disaster/open_images,1,1
20398,0143bd9e146f6ded.jpg,../Data/disaster/test/non_disaster/open_images,1,1
